In [1]:
#import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle
from sklearn import svm

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [2]:
filename_train ='/home/octo/Dropbox'+ '/SPY10Apr.csv'

In [1]:
# loading csv file
def get_csv_pd(path):
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    return spy_pd
def BA(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df=df.dropna()
    return df
def processing(df):
    df['Open']=(df.askPrice+df.bidPrice)/2
    df['Close']=df.Open.shift(-1)
    df['High']=df.askPrice.rolling(6).max()
    df['Low']=df.bidPrice.rolling(6).min()
    df['Volume']=df.askSize+df.bidSize
    df['vwap']=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df=df.dropna()
    return df
def tech_ind(data):
    data['km']=ml.kalman_ma(data)
    data['rsi']=talib.RSI(np.array(data.Close.astype('float64')),timeperiod=6)
    data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype  ('float64')),timeperiod=6)
    data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=6)# mom consists 0
    return data


In [28]:
data=get_csv_pd(filename_train)
data=BA(data)
data=processing(data)
data=tech_ind(data)
data=data.drop(['Stock'], axis=1)
data=(data-data.rolling(6).mean())
data['UD']=np.where(np.logical_and(data.Close.diff(6)>0.005,data.askPrice>data.km*1.001),1,np.where(np.logical_and(data.Close.diff(6)<-0.005,data.bidPrice<data.km*0.999),-1,0))
data['U']=np.where(np.logical_and(data.Close.diff(6)>0.005,data.askPrice>data.km*1.001),1,0)
data['D']=np.where(np.logical_and(data.Close.diff(6)<-0.005,data.bidPrice<data.km*0.999),-1,0)
data=data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom','UD','U','D']]
data=data.astype('float64')
data=data.dropna()

In [7]:
#len(data)

In [31]:
#data.head()

In [ ]:
#data.tail()

In [29]:
print(data.groupby('UD').size())

UD
-1.00000    15445
0.00000     46288
1.00000     11622
dtype: int64


## state analysis

In [32]:
from hmmlearn.hmm import GaussianHMM
print("fitting to HMM and decoding ...", end="")
# Make an HMM instance and execute fit
model_hmm = GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)
#model_hmm.fit(data[['Volume','vwap','km','rsi', 'atr','mom']])
model_hmm.fit(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])

fitting to HMM and decoding ...

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=1000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [33]:
from sklearn.externals import joblib
joblib.dump(model_hmm, "markov_hmm.pkl")

['markov_hmm.pkl']

## SMOTE

In [37]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state=14, ratio ='minority')
X_o,Y_o = sm.fit_sample(X,Y)

## Bayes

In [44]:
import numpy as np
import sklearn
from sklearn.naive_bayes import BernoulliNB
#from sklearn.naive_bayes import GaussianNB

In [45]:
X=data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']]
Yu=data.U
Yd=data.D

### UP

In [46]:
sm = SMOTE(random_state=14, ratio ='minority')
X_o,Y_o = sm.fit_sample(X,Yu)

In [47]:
bnb = BernoulliNB()
bnb.fit(X_o, Y_o)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [49]:
filename_nb = 'nbU.sav'
pickle.dump(bnb, open(filename_nb, 'wb'))

### Dn

In [50]:
sm = SMOTE(random_state=15, ratio ='minority')
X_o,Y_o = sm.fit_sample(X,Yd)

In [51]:
bnb = BernoulliNB()
bnb.fit(X_o, Y_o)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [52]:
filename_nb = 'nbD.sav'
pickle.dump(bnb, open(filename_nb, 'wb'))

## outlier

In [55]:
#outlier = svm.OneClassSVM(nu=0.05, kernel="rbf", gamma=0.1)
outlier = svm.OneClassSVM(kernel="poly", degree=3, gamma="auto", coef0=4.0, tol=0.001, nu=0.05, shrinking=True, cache_size=200, verbose=False, max_iter=-1, random_state=None)
outlier.fit(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])

OneClassSVM(cache_size=200, coef0=4.0, degree=3, gamma='auto', kernel='poly',
      max_iter=-1, nu=0.05, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [56]:
filename_outlr = 'outlr.sav'
pickle.dump(outlier, open(filename_outlr, 'wb'))

## RF for UD

In [57]:
X=data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']]
Y=data.UD

In [58]:
sm = SMOTE(random_state=11, ratio ='minority')
X_o,Y_o = sm.fit_sample(X,Y)

In [59]:
SEED=15
rf = RandomForestClassifier(
    n_estimators=9,
    max_features=6,
    random_state=SEED
)

rf.fit(X_o,Y_o)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=1,
            oob_score=False, random_state=15, verbose=0, warm_start=False)

In [60]:
filename_rf = 'rf.sav'
pickle.dump(rf, open(filename_rf, 'wb'))

In [61]:
# Kfold validation
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

kfold = model_selection.KFold(n_splits=20, random_state=SEED)
#results = model_selection.cross_val_score(rf, X_o,Y_o, cv=kfold)
results = model_selection.cross_val_score(rf, X,Y, cv=kfold)
print(results.mean())

0.9627968538995711


## Gradient Boosting for UD

In [62]:
# Stochastic Gradient Boosting Classification
import pandas
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier

In [63]:
seed = 7
num_trees = 9
#kfold = model_selection.KFold(n_splits=20, random_state=seed)
gb = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
#results = model_selection.cross_val_score(model, X, Y, cv=kfold)
#print(results.mean())
gb.fit(X,Y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=9, presort='auto',
              random_state=7, subsample=1.0, verbose=0, warm_start=False)

In [64]:
filename_gb = 'gb.sav'
pickle.dump(gb, open(filename_gb, 'wb'))